In [4]:
import numpy as np
from scipy.sparse import spdiags

with open("Fmat.npz", "r") as f:
    Fmat = np.load('Fmat.npz')['Fmat']
with open("permvec.npz", "r") as f:
    permvec = np.load('permvec.npz')['permvec']

In [12]:
# Problem 1
n = 8
L = 20.0
dx = L / n
N = n * n

e = np.ones(n)
D2 = spdiags([e, -2*e, e], [-1, 0, 1], n, n).toarray()
D2[0, -1] = 1.0
D2[-1, 0] = 1.0
D2 = D2 / dx**2

D1 = spdiags([-e, e], [-1, 1], n, n).toarray()
D1[0, -1] = -1.0
D1[-1, 0] =  1.0
D1 = D1 / (2.0 * dx)

A1 = np.zeros((N, N))
A2 = np.zeros((N, N))
A3 = np.zeros((N, N))

for i in range(n):
    for j in range(n):
        k = j + i * n
        for p in range(n):
            A1[k, j + p * n] += D2[i, p]
            A2[k, j + p * n] += D1[i, p]
        for q in range(n):
            A1[k, q + i * n] += D2[j, q]
            A3[k, q + i * n] += D1[j, q]


In [15]:
# Problem 2
perm = np.asarray(permvec).ravel().astype(int)
if perm.max() == 16 or perm.min() == 1:
    perm = perm - 1

N = Fmat.shape[0]
grid = 4
blk  = 20
side = grid * blk
mid  = N // 2
r0   = mid - side // 2
r1   = r0 + side
c0, c1 = r0, r1

center = Fmat[r0:r1, c0:c1].copy()
blocks = center.reshape(grid, blk, grid, blk).swapaxes(1, 2)
new_blocks = np.empty_like(blocks)

for k, src in enumerate(perm):
    di, dj = divmod(k,   grid)  
    si, sj = divmod(src, grid) 
    new_blocks[di, dj] = blocks[si, sj]

F_dec = Fmat.copy()
F_dec[r0:r1, c0:c1] = new_blocks.swapaxes(1, 2).reshape(side, side)

A4 = np.abs(F_dec)

F_unshift = np.fft.ifftshift(F_dec)
img_rec   = np.fft.ifft2(F_unshift)
A5 = np.abs(img_rec)